In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
from scipy import signal
from scipy.stats.stats import pearsonr

In [2]:
## Load Dataset
targets = []
keep_channels = ['Timestamp', 'EEG.Counter', 'EEG.P7', 'EEG.O1', 'EEG.O2', 'EEG.P8']
channels = ['EEG.O1', 'EEG.O2', 'EEG.P7', 'EEG.P8']

for i in range(8):
    df = pd.read_csv('recordings/checkerboard_recording_sultan_{}.csv'.format(i), skiprows=0)
    targets.append(df)

In [3]:
# cutoff according to shortest length
cutoff = min([len(targets[i].index) for i in range(8)])
for i in range(8):
    keep_indicies = set(range(targets[i].shape[0])) - set(range(cutoff, targets[i].shape[0]))
    targets[i] = targets[i].take(list(keep_indicies))
    print('length of dataset {} = {}'.format(i+1, len(targets[i].index)))

length of dataset 1 = 1920
length of dataset 2 = 1920
length of dataset 3 = 1920
length of dataset 4 = 1920
length of dataset 5 = 1920
length of dataset 6 = 1920
length of dataset 7 = 1920
length of dataset 8 = 1920


In [4]:
# add EEG.Counter column - 0 to 128 counter
seq = [i % 129 for i in range(len(targets[0].index))]
for target in targets:
    target.rename(columns = {'Unnamed: 0': 'Counter'}, inplace=True)
    target['Counter'] = seq

In [8]:
def filter_template(template, offset_mode='average'):
    channels = ['P7', 'O1', 'O2', 'P8']
    filt = signal.firwin(400, [0.01, 0.06], pass_zero=False)
    for channel in channels:
        if offset_mode == 'average':
            offset = np.average(np.array(template[channel]))
        elif offset_mode == 'minimum':
            offset = np.amin(np.array(template[channel]))
        elif offset_mode == 'same':
            offset = 3800
        elif offset_mode == 'none':
            offset = 0
        else:
            raise ValueError('choose a valid offset_mode')
        template[channel] = [i - offset for i in template[channel]]
        template[channel] = signal.convolve(template[channel], filt, mode='same')
        
    

In [9]:
for target in targets:
    filter_template(target)

In [10]:
targets[0]

,Counter,P7,O1,O2,P8,TIME
0,0,-9.561456,-3.343435,-4.003729,-3.802192,1.603039e+09
1,1,-11.409814,-3.897338,-4.832102,-3.935986,1.603039e+09
2,2,-13.156168,-4.396484,-5.601704,-4.039875,1.603039e+09
3,3,-14.767379,-4.828228,-6.294223,-4.105408,1.603039e+09
4,4,-16.213074,-5.181002,-6.892716,-4.125112,1.603039e+09
...,...,...,...,...,...,...
1915,109,0.655595,1.996756,1.661130,-2.542917,1.603039e+09
1916,110,0.688208,1.771492,1.111060,-1.634517,1.603039e+09
1917,111,0.673235,1.483182,0.529890,-0.705806,1.603039e+09
1918,112,0.613982,1.138518,-0.074955,0.232227,1.603039e+09
